<a href="https://colab.research.google.com/github/KushanNalinka/Transformer_neural_network/blob/main/TransformerApplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
!unzip -q spa-eng.zip

--2023-09-03 08:46:37--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.31.128, 142.251.18.128, 142.250.153.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.31.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2638744 (2.5M) [application/zip]
Saving to: ‘spa-eng.zip’

spa-eng.zip         100%[===================>]   2.52M  3.92MB/s    in 0.6s    

2023-09-03 08:46:38 (3.92 MB/s) - ‘spa-eng.zip’ saved [2638744/2638744]



In [ ]:
text_file = "spa-eng/spa.txt"
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    english, spanish = line.split("\t")
    spanish = "[start] " + spanish + " [end]"
    text_pairs.append((english, spanish))

In [ ]:
import random
print(random.choice(text_pairs))

('Tom was killed by a suicide bomber.', '[start] A Tom le mató un kamikaze. [end]')


In [ ]:
import random
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

In [ ]:
import tensorflow as tf
import string
import re
from tensorflow import keras
from tensorflow.keras import layers

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

In [ ]:
batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        "english": eng,
        "spanish": spa[:, :-1],
    }, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [ ]:
print(list(train_ds.as_numpy_iterator())[50]);

({'english': array([[ 108,  764, 2985, ...,    0,    0,    0],
       [   3,  270,    2, ...,    0,    0,    0],
       [  13,  371,  356, ...,    0,    0,    0],
       ...,
       [  24,   50,    6, ...,    0,    0,    0],
       [  21,    8,   34, ...,    0,    0,    0],
       [   3,  575,   75, ...,    0,    0,    0]]), 'spanish': array([[  2,  35,  74, ...,   0,   0,   0],
       [  2,  35, 691, ...,   0,   0,   0],
       [  2,  71,   5, ...,   0,   0,   0],
       ...,
       [  2,  27, 125, ...,   0,   0,   0],
       [  2,  26,   7, ...,   0,   0,   0],
       [  2,  14, 554, ...,   0,   0,   0]])}, array([[  35,   74, 2104, ...,    0,    0,    0],
       [  35,  691,    5, ...,    0,    0,    0],
       [  71,    5, 1576, ...,    0,    0,    0],
       ...,
       [  27,  125,    8, ...,    0,    0,    0],
       [  26,    7,   12, ...,    0,    0,    0],
       [  14,  554,   16, ...,    0,    0,    0]]))


In [ ]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=(None,), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(
    layers.GRU(latent_dim), merge_mode="sum")(x)

In [ ]:
past_target = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

In [ ]:
seq2seq_rnn.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 english (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 spanish (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 256)    3840000     ['english[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 256)    3840000     ['spanish[0][0]']                
                                                                                              

In [ ]:
seq2seq_rnn.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds)

Epoch 1/15
1302/1302 [==============================] - 144s 96ms/step - loss: 4.6825 - accuracy: 0.3177 - val_loss: 3.9098 - val_accuracy: 0.3855
Epoch 2/15
1302/1302 [==============================] - 106s 82ms/step - loss: 3.7319 - accuracy: 0.4138 - val_loss: 3.2587 - val_accuracy: 0.4682
Epoch 3/15
1302/1302 [==============================] - 106s 82ms/step - loss: 3.2207 - accuracy: 0.4714 - val_loss: 2.8877 - val_accuracy: 0.5157
Epoch 4/15
1302/1302 [==============================] - 106s 81ms/step - loss: 2.8652 - accuracy: 0.5122 - val_loss: 2.6356 - val_accuracy: 0.5498
Epoch 5/15
1302/1302 [==============================] - 106s 81ms/step - loss: 2.5978 - accuracy: 0.5439 - val_loss: 2.4589 - val_accuracy: 0.5754
Epoch 6/15
1302/1302 [==============================] - 105s 81ms/step - loss: 2.3693 - accuracy: 0.5730 - val_loss: 2.3294 - val_accuracy: 0.5944
Epoch 7/15
1302/1302 [==============================] - 106s 81ms/step - loss: 2.1815 - accuracy: 0.5971 - val_loss: 2

In [ ]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

In [ ]:
test_eng_texts = [pair[0] for pair in test_pairs]


In [ ]:
len(test_eng_texts)

17844

In [ ]:
input_sentence = random.choice(test_eng_texts);
print(input_sentence)

I've never felt so exhilarated.


In [ ]:
tokenized_input_sentence = source_vectorization([input_sentence])
print(tokenized_input_sentence)

tf.Tensor(
[[108  97 439  84   1   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]], shape=(1, 20), dtype=int64)


In [ ]:
def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])
        next_token_predictions = seq2seq_rnn.predict(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(next_token_predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
Everybody needs something to believe in.
1/1 [==============================] - 0s 26ms/step
[start] todo el mundo lo que debe hablar en el lugar [end]
-
You can't fix it. You should buy a new one.
1/1 [==============================] - 0s 28ms/step
[start] no puedes creer que hayas comprado algo nuevo [end]
-
You had better take an umbrella with you.
1/1 [==============================] - 0s 25ms/step
[start] mejor hubieras mejor te llevo un paraguas [end]
-
I need a vacation.
1/1 [==============================] - 0s 24ms/step
[start] necesito unas vacaciones [end]
-
She waited for him for hours.
1/1 [==============================] - 0s 25ms/step
[start] ella lo esperó por horas [end]
-
That's a problem.
1/1 [==============================] - 0s 23ms/step
[start] ese es un problema [end]
-
Everyone is waiting to see his new film.
1/1 [==============================] - 0s 23ms/step
[start] todos están a punto de ver su nuevo película [end]
-
I shot Tom.
1/1 [=======================

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [ ]:

class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

In [ ]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [ ]:
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [32]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30
1302/1302 [==============================] - 107s 77ms/step - loss: 1.6300 - accuracy: 0.7614 - val_loss: 2.4482 - val_accuracy: 0.6698
Epoch 2/30
1302/1302 [==============================] - 84s 64ms/step - loss: 1.6058 - accuracy: 0.7654 - val_loss: 2.4484 - val_accuracy: 0.6723
Epoch 3/30
1302/1302 [==============================] - 87s 67ms/step - loss: 1.5995 - accuracy: 0.7665 - val_loss: 2.4997 - val_accuracy: 0.6669
Epoch 4/30
1302/1302 [==============================] - 85s 66ms/step - loss: 1.5895 - accuracy: 0.7683 - val_loss: 2.5126 - val_accuracy: 0.6618
Epoch 5/30
1302/1302 [==============================] - 83s 64ms/step - loss: 1.5787 - accuracy: 0.7700 - val_loss: 2.4951 - val_accuracy: 0.6680
Epoch 6/30
1302/1302 [==============================] - 84s 65ms/step - loss: 1.5638 - accuracy: 0.7726 - val_loss: 2.5266 - val_accuracy: 0.6667
Epoch 7/30
1302/1302 [==============================] - 84s 65ms/step - loss: 1.5506 - accuracy: 0.7751 - val_loss: 2.5071 

In [33]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
We'll come and visit you.
[start] iremos y visitar [end]
-
A lot of people need our help.
[start] mucha gente nuestra necesario es la ayuda [end]
-
"May I park here?" "No, you can't."
[start] puedo estacionar aquí no no puede ser [end]
-
The Japanese tend to identify themselves with the group they belong to.
[start] los japoneses [UNK] con [UNK] la banda de cuchillo [end]
-
The color white is a symbol of purity.
[start] el color de blanco es un misterio [end]
-
I feel like going on a trip.
[start] me gusta ir de viaje [end]
-
It's about time.
[start] ya casi es hora [end]
-
He waited for several seconds and opened the door.
[start] Él es la esperando varios problemas y abrió la puerta [end]
-
You don't want that, do you?
[start] no quieres hacerlo verdad [end]
-
I doubt that Tom has the courage to do what really needs to be done.
[start] yo dudo que tom haya [UNK] el valor de hacer realmente que una posibilidad debe hacer [end]
-
Tom didn't want me to touch him.
[start] tom no me que